## Module imports

In [ ]:
import keras
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense
from keras.models import Model

from keras.preprocessing.image import ImageDataGenerator
from PIL import Image

## Parameters for training

In [ ]:
epochs = 50
batch_size = 16

# Number of training samples
number_of_images = 800
steps_per_epoch = number_of_images // batch_size

# Number of categories
classes = 4

## Image augmentations

### Traning augmentations

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
rescale=1./255)

### Validation set augmentations

In [ ]:
validation_datagen = ImageDataGenerator(
    rescale=1./255)

## Initialize VGG16 base model and weights

In [ ]:
base_model= VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))

## Adding top model to base model

In [ ]:
top_model = Flatten()(base_model.output)
top_model = Dense(256, activation='relu')(top_model)
top_model = Dense(classes, activation='softmax')(top_model)

## Instantiate the functional model object 

In [ ]:
model = Model(inputs=base_model.input, outputs=top_model)

## Compile model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

## Lock all convolution and pooling layers

In [ ]:
for layer in model.layers[:20]:
    layer.trainable = False

In [ ]:
model.summary()

## Load datasets using data generators

In [ ]:
train_generator = train_datagen.flow_from_directory(
        # Add training directory
        'train',
        target_size=(150, 150),
        batch_size=batch_size,
    
        # 'binary' for 2 classes, 'categorical' > 2 classes
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        # Add validation directory
        'validation',
        target_size=(150, 150),
        batch_size=batch_size,
    
        # 'binary' for 2 classes, 'categorical' > 2 classes
        class_mode='categorical')


## Train the model

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=200)

## Save model weights

In [ ]:
model.save_weights('model.h5')

## Try to predict an image using the trained model

In [ ]:
# path to image for prediction
img_for_prediction = Image.open("path")

# Transform to numpy array
img_for_prediction = np.array(img_for_prediction)

# Reshape (150,150,3) to (1,150,150,3)
img_for_prediction = img_for_prediction[np.newaxis,...]

# Predict
model.predict(img_for_prediction)